<a href="https://colab.research.google.com/github/NikolasGialitsis/PredictingFootballMatches/blob/master/PredictingFootballOutcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [183]:
!git clone https://github.com/footballcsv/greece.git


fatal: destination path 'greece' already exists and is not an empty directory.


In [184]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

main_dir = 'greece'
filename='gr.1.csv'
directories = [x[0] for x in os.walk(main_dir) if '-' in x[0]]
print(directories)
print('Number of yearly collections:',len(directories))
YearArchives = [dir + '/' + filename for dir in directories]

total=0
df = pd.DataFrame()
for id in range(len(YearArchives)):
  arch = YearArchives[id]
  if total == 0:
    df = pd.read_csv(arch)
  else:
    new_df = pd.read_csv(arch)
    df = pd.concat([df,new_df],ignore_index=True)
  print(directories[id],' contains ', df.shape[0] - total, ' football matches')
  total = df.shape[0]

print('total football matches in greece : ',total)
print(df.columns)
print(df.head())


['greece/2000-01', 'greece/2002-03', 'greece/2019-20', 'greece/2005-06', 'greece/1998-99', 'greece/2009-10', 'greece/2007-08', 'greece/2017-18', 'greece/2003-04', 'greece/2012-13', 'greece/2006-07', 'greece/2001-02', 'greece/2004-05', 'greece/2013-14', 'greece/2014-15', 'greece/1995-96', 'greece/2010-11', 'greece/2011-12', 'greece/1996-97', 'greece/1999-00', 'greece/2008-09', 'greece/2018-19', 'greece/2016-17', 'greece/1994-95', 'greece/1997-98', 'greece/2015-16']
Number of yearly collections: 26
greece/2000-01  contains  240  football matches
greece/2002-03  contains  240  football matches
greece/2019-20  contains  14  football matches
greece/2005-06  contains  240  football matches
greece/1998-99  contains  306  football matches
greece/2009-10  contains  240  football matches
greece/2007-08  contains  240  football matches
greece/2017-18  contains  240  football matches
greece/2003-04  contains  240  football matches
greece/2012-13  contains  240  football matches
greece/2006-07  con

In [185]:
# Drop 'Round' column
df = df.drop(labels = ['Round'], axis = 1)
print(df.columns)


Index(['Date', 'Team 1', 'FT', 'HT', 'Team 2'], dtype='object')


In [186]:
def count_missing_values(df):
  
  for col in df.columns:
    qmarks = df[df[col].str.contains("\?")]
    if(~qmarks.empty):
      print('\t',col,':',len(qmarks))


#check for null entries
assert(True not in df.isnull().any())


#check for missing values
print('\nMissing values initially')
count_missing_values(df)


#try to fill in missing values, between HT and FT
missingFT = df[df['FT'].str.contains("\?")].index
missingHT = df[df['HT'].str.contains("\?")].index

filled=0
for index in missingFT:
  if index not in missingHT:
    df.iloc[index]['FT'] = df.iloc[index]['HT']
    filled+=1
for index in missingHT:
  if index not in missingFT:
    df.iloc[index]['HT'] = df.iloc[index]['FT']
    filled+=1

#after correction
print('\nMissing values after filling-in %d row(s)'%filled)
count_missing_values(df)

#drop any remaining rows with missing values
missingFT = df[df['FT'].str.contains("\?")].index
df = df.drop(missingFT)
print('\nMissing values after dropping the %d non-informative row(s)'%len(missingFT))
count_missing_values(df)


Missing values initially
	 Date : 0
	 Team 1 : 0
	 FT : 2
	 HT : 1559
	 Team 2 : 0

Missing values after filling-in 1559 row(s)
	 Date : 0
	 Team 1 : 0
	 FT : 1
	 HT : 1
	 Team 2 : 0

Missing values after dropping the 1 non-informative row(s)
	 Date : 0
	 Team 1 : 0
	 FT : 0
	 HT : 0
	 Team 2 : 0


In [187]:
#ignore parentheses on team names
import re
for index, row in df.iterrows():
  team1 = row['Team 1'] 
  team2 = row['Team 2']
  
  row['Team 1'] = re.sub('\(\d*\)','',team1)
  row['Team 2'] = re.sub('\(\d*\)','',team2)
  if index == 0:
    print(team1)
    print('\t',row['Team 1'])
    print(team2)
    print('\t',row['Team 2'])
  

Olympiacos Piraeus FC (1)
	 Olympiacos Piraeus FC 
Ethnikos Asteras FC (1)
	 Ethnikos Asteras FC 


In [188]:
#repeat all matches in the dataframe, but in reverse
#ignore parentheses on team names
copy_df = df.copy()
assert(copy_df.shape[0] == df.shape[0])
for index, row in copy_df.iterrows():
  team1 = row['Team 1'] 
  team2 = row['Team 2']
  FT = row['FT']
  HT = row['HT']

  #reverse teams
  row['Team 1'] = team2
  row['Team 2'] = team1

  #reverse final score
  tokens = FT.split('-')
  row['FT'] = str(tokens[1] + '-' + tokens[0]) 

  #reverse half time score
  tokens = HT.split('-')
  row['HT'] = str(tokens[1] + '-' + tokens[0]) 

df = pd.concat([df,copy_df],ignore_index=True)
print('New merged dataframe',df.shape)

New merged dataframe (12966, 5)


In [209]:
team_df = df.groupby('Team 1')

for v in team_df['Team 1']:
    print(v)


('AE Larissa FC ', 491      AE Larissa FC 
509      AE Larissa FC 
521      AE Larissa FC 
549      AE Larissa FC 
559      AE Larissa FC 
              ...      
12365    AE Larissa FC 
12367    AE Larissa FC 
12378    AE Larissa FC 
12398    AE Larissa FC 
12418    AE Larissa FC 
Name: Team 1, Length: 340, dtype: object)
('AEK Athens FC ', 6        AEK Athens FC 
17       AEK Athens FC 
41       AEK Athens FC 
62       AEK Athens FC 
73       AEK Athens FC 
              ...      
12903    AEK Athens FC 
12924    AEK Athens FC 
12937    AEK Athens FC 
12943    AEK Athens FC 
12960    AEK Athens FC 
Name: Team 1, Length: 712, dtype: object)
('AEL Kalloni FC ', 2910     AEL Kalloni FC 
2920     AEL Kalloni FC 
2954     AEL Kalloni FC 
2969     AEL Kalloni FC 
2987     AEL Kalloni FC 
              ...       
12891    AEL Kalloni FC 
12904    AEL Kalloni FC 
12923    AEL Kalloni FC 
12939    AEL Kalloni FC 
12961    AEL Kalloni FC 
Name: Team 1, Length: 98, dtype: object)
('APO Akratito